In [2]:
import pandas as pd
import sys, os
sys.path.insert(0, os.path.abspath('../dolphyn'))
import dolphyn as D
import imp

In [3]:
imp.reload(D)

<module 'dolphyn' from '/home/anna/project/phageomeLib/Public/dolphyn/dolphyn.py'>

In [4]:
hfc = pd.read_table("../data/hfc_pubEpitopes.csv", index_col = 0, sep = ",")

In [5]:
for k in [15,20,25,30,35,40,45]:
    feat = pd.DataFrame(columns = D.all_feature_names())
    labs = pd.DataFrame(columns=["reactivity", "reactivity_binary"])
    c = 0
    mersuffix = "_" + str(k) + "mer"
    print(mersuffix)
    for index, row in hfc.iterrows():      
        if mersuffix in index :  
            if c%500 == 0:
                print(str(c), "sequences done")
            c += 1
            seq = index.split("_")[-1][-k:]
            labs.loc[index, "reactivity"] = row.sum()
            labs.loc[index, "reactivity_binary"] = 1 if sum(row > 0) > 1 else 0   
            feat.loc[index] = D.kmer_features_of_protein(seq,k).loc[0]
    feat.to_csv("peds_features"+mersuffix+".csv")
    labs.to_csv("peds_labels"+mersuffix+".csv")

_15mer
0 sequences done
500 sequences done
1000 sequences done
1500 sequences done
2000 sequences done
2500 sequences done
3000 sequences done
_20mer
0 sequences done
500 sequences done
1000 sequences done
1500 sequences done
2000 sequences done
2500 sequences done
3000 sequences done
_25mer
0 sequences done
500 sequences done
1000 sequences done
1500 sequences done
2000 sequences done
2500 sequences done
_30mer
0 sequences done
500 sequences done
1000 sequences done
1500 sequences done
2000 sequences done
_35mer
0 sequences done
500 sequences done
1000 sequences done
1500 sequences done
2000 sequences done
_40mer
0 sequences done
500 sequences done
1000 sequences done
1500 sequences done
_45mer
0 sequences done
500 sequences done
1000 sequences done


In [2]:
MY_SEED = 55

import os
import time
import json
import random
import pandas as pd
import numpy as np
import math
import matplotlib
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_curve
from sklearn.metrics import average_precision_score

def file_len(fname):
    with open(fname) as f:
        for i, l in enumerate(f):
            pass
    return i + 1

def file_head(file, l=10):
    with open(file) as myfile:
        head_repres = [next(myfile) for x in range(l)]
    return(head_repres)

def execute(command):
    stream = os.popen(command)
    output = stream.read().strip()
    return(output)

def isNaN(num):
    return num != num



def TFPN(ground_truth, prediction):
    TFPN = ""
    if ground_truth:
        if prediction:
            TFPN = "TP"
        else:
            TFPN = "FN"
    else:
        if prediction:
            TFPN = "FP"
        else:
            TFPN = "TN"
    return(TFPN)


# Create Training DS and test features

In [23]:
def create_DS(allfeatures, aa, d_aa, aa_double, d_aa_double, side_chain, verbose = True):
    X = allfeatures
    if (aa == False):
        X = X.drop(X.columns[[len(colname) == 1 for colname in X.columns]], axis = 1)
    else:
        if verbose:
            print("include AA features in dataset")
    if (aa_double == False):
        X = X.drop(X.columns[[len(colname) == 2 for colname in X.columns]], axis = 1)
    else:
        if verbose:
            print("include aa_double features in dataset")
    if (d_aa == False):
        X = X.drop(X.columns[[len(colname) == 3 for colname in X.columns]], axis = 1)
    else:
        if verbose:
            print("include d_aa features in dataset")
    if (d_aa_double == False):
        X = X.drop(X.columns[[len(colname) == 4 for colname in X.columns]], axis = 1)
    else:
        if verbose:
            print("include d_aa_double features in dataset")
    if (side_chain == False):
        X = X.drop(X.columns[["sc_" in colname for colname in X.columns]], axis = 1)
    else:
        if verbose:
            print("include side_chain features in dataset")
    return(X)

def train_test_split_15merIndex(X, y, test_size, random_state, balanced=False):
    wildtypeIDs = set([item[0] for item in X.index.str.split("_")])
    random.seed(random_state)
    
    # first split train/test set, then balance the training set!   
    test_IDs = random.sample(wildtypeIDs, int(test_size*len(wildtypeIDs)))
    train_IDs = wildtypeIDs.difference(test_IDs)
    X_train = X.iloc[[item[0] in train_IDs for item in X.index.str.split("_")],:]
    X_test = X.iloc[[item[0] in test_IDs for item in X.index.str.split("_")],:]
    y_train = y.iloc[[item[0] in train_IDs for item in y.index.str.split("_")]]
    y_test = y.iloc[[item[0] in test_IDs for item in y.index.str.split("_")]]
    
    
    if balanced:
        size_smaller_group = y_train.value_counts().min()
        pos_IDs = set(y_train[y_train==1].index)
        neg_IDs = set(y_train[y_train==0].index)
        neg_IDs= random.sample(neg_IDs, size_smaller_group)
        pos_IDs= random.sample(pos_IDs, size_smaller_group)
        trainIDs = set(pos_IDs + neg_IDs)
        y_train = y_train[trainIDs]
        X_train = X_train.loc[trainIDs,]
    
    return(X_train, X_test, y_train, y_test)

In [33]:
X = create_DS(allfeatures = feat, aa=False, d_aa=False, aa_double=True, d_aa_double=False, side_chain=True)
y = labs["reactivity_binary"].astype('int')
y.value_counts()
X

include aa_double features in dataset
include side_chain features in dataset


,sc_hydrophobic,sc_polaruncharged,sc_poseleccharged,sc_negeleccharged,AA,AR,AN,AD,AC,AQ,AE,AG,AH,AI,AL,AK,AM,AF,AP,AS,AT,AW,AY,AV,RA,...,YS,YT,YW,YY,YV,VA,VR,VN,VD,VC,VQ,VE,VG,VH,VI,VL,VK,VM,VF,VP,VS,VT,VW,VY,VV
52925_15mer_start_1_GGGGSSPPRRPPPGRRPFFH,2,1,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
52925_15mer_start_6_GGGGSPPPGRRPFFHPVAEA,5,0,3,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
52925_15mer_start_11_GGGGSRPFFHPVAEADYFEY,8,0,2,3,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
52925_15mer_start_16_GGGGSPVAEADYFEYHQEGG,6,1,1,4,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
52925_15mer_start_21_GGGGSDYFEYHQEGGPDGEP,3,1,1,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45395_15mer_start_26_GGGGSLSTRYGNVTIPFVSP,6,5,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0
45395_15mer_start_31_GGGGSGNVTIPFVSPVDVPL,7,3,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0
45395_15mer_start_36_GGGGSPFVSPVDVPLHTGPD,5,2,1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0
45395_15mer_start_41_GGGGSVDVPLHTGPDIILPS,6,2,1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [34]:
# all configurations
test_size = 0.2
#balanced = True

eval_df = pd.DataFrame()
i = 0
for aa in [True,False]:
    for d_aa in [True,False]:
        for aa_double in [True,False]:
            for d_aa_double in [True,False]:
                for side_chain in [True,False]:
                    for randomstate_tt in [60,10]:
                        for randomstate_rf in [42,123]:
                            for balanced in [True]:
                                if(aa or d_aa or aa_double or d_aa_double or side_chain):                    
                                    print("config " + str(i))
                                    X = create_DS(allfeatures = feat, aa=aa, d_aa=d_aa, aa_double=aa_double, d_aa_double=d_aa_double, side_chain=side_chain, verbose = False)
                                    X_train, X_test, y_train, y_test = train_test_split_15merIndex(X, y, test_size=test_size, random_state = randomstate_tt, balanced = balanced) 
                                    #print('Testing Features Shape:', X_test.shape)
                                    clf=RandomForestClassifier(n_estimators=100, n_jobs=6, random_state=randomstate_rf)
                                    clf.fit(X_train,y_train)
                                    y_pred=clf.predict(X_test)
                                    #print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
                                    #print("Confusion Matrix:\n",metrics.confusion_matrix(y_test, y_pred))
                                    feature_imp = pd.Series(clf.feature_importances_,index=X.columns).sort_values(ascending=False)
                                    #print("Top 5 features\n",feature_imp[:5], "\n")

                                    #eval_df.loc[i, "balanced_train"] = balanced
                                    eval_df.loc[i, "random_state_tt"] = randomstate_tt
                                    eval_df.loc[i, "random_state_rf"] = randomstate_rf
                                    eval_df.loc[i, "no_features"] = X_test.shape[1]
                                    eval_df.loc[i, "aa"] = aa
                                    eval_df.loc[i, "d_aa"] = d_aa
                                    eval_df.loc[i, "aa_double"] = aa_double
                                    eval_df.loc[i, "d_aa_double"] = d_aa_double
                                    eval_df.loc[i, "side_chain"] = side_chain
                                    eval_df.loc[i, "Accuracy"] = metrics.accuracy_score(y_test, y_pred)
                                    eval_df.loc[i, "AUC"] = metrics.roc_auc_score(y_test, y_pred)
                                    eval_df.loc[i, "Precision"] = metrics.precision_score(y_test, y_pred)
                                    eval_df.loc[i, "Recall"] = metrics.recall_score(y_test, y_pred)
                                    eval_df.loc[i, "F1"] = metrics.f1_score(y_test, y_pred)
                                    eval_df.loc[i, "Top1"] = feature_imp.index[0]
                                    eval_df.loc[i, "Top2"] = feature_imp.index[1]
                                    eval_df.loc[i, "Top3"] = feature_imp.index[2]
                                    eval_df.loc[i, "Top4"] = feature_imp.index[3]
                                    if len(feature_imp.index) > 4:
                                        eval_df.loc[i, "Top5"] = feature_imp.index[4]
                                    #eval_df.loc[i, "CM"] = metrics.confusion_matrix(y_test, y_pred)
                                    eval_df.loc[i, "TP"] = metrics.confusion_matrix(y_test, y_pred)[1][1]
                                    eval_df.loc[i, "TN"] = metrics.confusion_matrix(y_test, y_pred)[0][0]
                                    eval_df.loc[i, "FP"] = metrics.confusion_matrix(y_test, y_pred)[0][1]
                                    eval_df.loc[i, "FN"] = metrics.confusion_matrix(y_test, y_pred)[1][0]
                                    i = i +1


config 0
config 1
config 2
config 3
config 4
config 5
config 6
config 7
config 8
config 9
config 10
config 11
config 12
config 13
config 14
config 15
config 16
config 17
config 18
config 19
config 20
config 21
config 22
config 23
config 24
config 25
config 26
config 27
config 28
config 29
config 30
config 31
config 32
config 33
config 34
config 35
config 36
config 37
config 38
config 39
config 40
config 41
config 42
config 43
config 44
config 45
config 46
config 47
config 48
config 49
config 50
config 51
config 52
config 53
config 54
config 55
config 56
config 57
config 58
config 59
config 60
config 61
config 62
config 63
config 64
config 65
config 66
config 67
config 68
config 69
config 70
config 71
config 72
config 73
config 74
config 75
config 76
config 77
config 78
config 79
config 80
config 81
config 82
config 83
config 84
config 85
config 86
config 87
config 88
config 89
config 90
config 91
config 92
config 93
config 94
config 95
config 96
config 97
config 98
config 99
config 100

In [10]:
eval_df.sort_values(by=['F1'], ascending=False)

NameError: name 'eval_df' is not defined

In [36]:
eval_eval_df = pd.DataFrame()
i = 0
for features in ["aa","d_aa","aa_double","d_aa_double","side_chain"]:
    for score in ["F1", "Recall", "Precision", "Accuracy", "AUC"]:
        eval_eval_df.loc[features, score+ "_mean"] = np.mean(eval_df[eval_df[features]==True][score])
        eval_eval_df.loc[features, score+ "_max"] = np.max(eval_df[eval_df[features]==True][score])
        eval_eval_df.loc[features, score+ "_min"] = np.min(eval_df[eval_df[features]==True][score])

In [37]:
eval_eval_df.sort_values(by=['F1_mean'], ascending=False)

,F1_mean,F1_max,F1_min,Recall_mean,Recall_max,Recall_min,Precision_mean,Precision_max,Precision_min,Accuracy_mean,Accuracy_max,Accuracy_min,AUC_mean,AUC_max,AUC_min
d_aa_double,0.461972,0.502024,0.394191,0.507304,0.553571,0.424107,0.424596,0.475610,0.368217,0.614945,0.655977,0.574344,0.587171,0.621550,0.535647
side_chain,0.460898,0.502024,0.420650,0.520915,0.569507,0.464286,0.414430,0.475610,0.349162,0.602112,0.655977,0.516788,0.581159,0.621550,0.528105
d_aa,0.460839,0.502024,0.421941,0.513442,0.569507,0.446429,0.418625,0.466926,0.374150,0.608222,0.648688,0.566423,0.583764,0.619589,0.547503
aa_double,0.460102,0.498960,0.394191,0.512690,0.569507,0.424107,0.417882,0.466926,0.368217,0.607859,0.648688,0.570803,0.583303,0.619589,0.535647
aa,0.456847,0.502024,0.394191,0.512966,0.569507,0.424107,0.412298,0.464000,0.364583,0.602299,0.647230,0.558309,0.579248,0.618777,0.535647
